[TOC](../toc.ipynb)

Advanced topics in nonlinear algebra
====================================

- KEYWORDS: scipy.optimize.fsolve, numpy.roots, numpy.polyder, numpy.polyval, numpy.polyint, numpy.poly1d




## Special nonlinear systems - polynomials





Polynomials are a special class of nonlinear algebraic equations that are especially easy to solve. A polynomial is linear in the coefficients in front of the variable. If we consider the following $n^{th}$ order polynomial:

$p_0 x^n + p_1 x^{(n-1)} + ... + p_{n-1} x + p_n = 0$

Let's be specific:

$x^2 + 8x + 16 = 0$

We express this as [1, 8, 16].



In [ ]:
import numpy as np
?np.roots



In [ ]:
p = [1, 8, 16]
r = np.roots(p)
r



In [ ]:
r**2 + 8 * r + 16



In [ ]:
import matplotlib.pyplot as plt

X = np.linspace(-5, -3)
y = X**2 + 8 * X + 16
plt.plot(X, y)
plt.xlabel('x')
plt.ylabel('y');



Note we get all the roots. We can check that with the `numpy.polyval` command.





In [ ]:
?np.polyval



In [ ]:
np.polyval(p, r)  # Another new command



We can also use this to plot a polynomial.





In [ ]:
import numpy as np

x = np.linspace(-5, -3)
y = np.polyval(p, x)

import matplotlib.pyplot as plt

plt.plot(x, y)
plt.xlabel("x")
plt.ylabel("y");



In [ ]:
from scipy.optimize import root


def f(x):
    return np.polyval(p, x)


root(f, -3.01)



Why is this so convenient?





### Cubic equations of state





There are applications of polynomials in thermodynamics. The van der waal equation is a cubic polynomial $f(V) = V^3 - \frac{p n b + n R T}{p} V^2 + \frac{n^2 a}{p}V - \frac{n^3 a b}{p} = 0$, where $a$ and $b$ are constants, $p$ is the pressure, $R$ is the gas constant, $T$ is an absolute temperature and $n$ is the number of moles. The roots of this equation tell you the volume of the gas at those conditions.





In [ ]:
# numerical values of the constants
a = 3.49e4
b = 1.45
p = 679.7  # pressure in psi
T = 683  # T in Rankine
n = 1.136  # lb-moles
R = 10.73  # ft^3 * psi / R / lb-mol

ppar = [
    1.0,  # V**3
    -(p * n * b + n * R * T) / p,  # V**2
    n**2 * a / p,  # V
    -(n**3) * a * b / p,
]  # constant, together this defines f(V)

print(np.roots(ppar))



In [ ]:
rr = np.roots(ppar)
(rr[0] * 2).imag



In [ ]:
V = np.linspace(0, 10)
plt.plot(V, np.polyval(ppar, V));



Note that only one root is real (and even then, we have to interpret 0.j as not being imaginary. Also, in a cubic polynomial, there can only be two imaginary roots). In this case that means there is only one phase present.

Remember fsolve? Two of these roots are not accessible in fsolve because they are imaginary.




### Other useful things to remember about polynomials





You can easily get the parameters of the derivative of the polynomial with `numpy.polyder`.





In [ ]:
?np.polyder



$y = x^2 + 8x + 16$


$y'(x) = 2x + 8$



In [ ]:
p = [1, 8, 16]

pd = np.polyder(p)  # New command
pd



You can use these with `numpy.polyval` to compute the derivative at different points. Note, this is an *analytical* derivative! No approximation necessary.





In [ ]:
np.polyval(pd, np.linspace(-5, -3))



You can also get the coefficients of the integral of the polynomial. The integration constant is assumed to be 0 by default.





In [ ]:
?np.polyint



In [ ]:
pint = np.polyint(p)  # new command
pint



You can use this to compute definite integrals, e.g. from x=1 to x=2:





In [ ]:
?np.polyval



In [ ]:
np.polyval(pint, 2) - np.polyval(pint, 1)



In [ ]:
from scipy.integrate import quad


def integrand(x):
    return x**2 + 8 * x + 16


quad(integrand, 1, 2)



In [ ]:
p



In [ ]:
X = np.linspace(1, 2)
y = X**2 + 8 * X + 16
from scipy.integrate import simps

simps(y, X)



**exercise** Use another method to confirm the result above.

Finally, the syntax `np.polyval(pint, 2)` can be a little tedious. You can create a function with `numpy.poly1d` using the array of coefficients. Conveniently, you can use the function in the roots, polyder and polyint commands!





In [ ]:
?np.poly1d



In [ ]:
p = np.poly1d(pint)
p(2) - p(1)



In [ ]:
p



In [ ]:
np.roots(p)



In [ ]:
?p



In [ ]:
print(p)



In [ ]:
p.r  # shorthand for roots



In [ ]:
[
    root.real for root in p.r if np.abs(root.imag) <= 1e-12
]  # extract out only the real part of the real roots.



In [ ]:
with np.printoptions(suppress=True):  # show very small numbers as 0
    print(p(p.r))



In [ ]:
p.coefficients



In [ ]:
p.order



## Systems of nonlinear equations





```{note}
There is also `scipy.optimize.fsolve` which can also be used to solve nonlinear equations. This is an older function, and `root` is preferred for new code.
```



Analogously to systems of ordinary differential equations, with systems of nonlinear equations we define functions that will return a zero for each equation in the system. Then we have to pass an initial guess for each variable to fsolve, and it will return an array of values, one for each variable.

It is considerably more difficult to visualize systems of nonlinear equations. With two equations and two unknowns it is sometimes easy to plot solutions, but not always.

\begin{align}
y &=& x^2 \\
y &=& 8 - x^2
\end{align}

One approach to visualizing this is to plot the two curves.





In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, 4)

y1 = x**2
y2 = 8 - x**2

plt.plot(x, y1, x, y2)
plt.xlabel("x")
plt.ylabel("y")
plt.legend(["y1", "y2"]);



You can see that on this domain, there is one place where the two curves intersect near the point (2, 5), which is a solution point. At this point there is one (x, y) pair that is a solution to *both* equations.



In [ ]:
from scipy.optimize import root


def objective(X):
    x, y = X
    z1 = y - x**2
    z2 = y - 8 + x**2
    return np.array([z1, z2])


guess = [2, 3.6]
objective(guess)



In [ ]:
ans = root(objective, guess)  

ans



In [ ]:
sol = root(objective, guess)  # we unpack the two parts of the solution into x and y
x, y = sol.x



In [ ]:
[
    objective((x, y)),  # tuple
    objective([x, y]),  # list
    # use an array
    objective(np.array([x, y])),
]  # three equivalent ways to call the objective function



In [ ]:
objective(ans.x)



In [ ]:
objective(guess)



## A harder example



It is not always easy to solve for one variable in terms of the other though. In that case, we can resort to an alternate graphical approach where we evaluate the objective function over a range of the variables, and look for regions where they overlap.

Consider the solution to these equations (adapted from [https://www.mathworks.com/help/optim/ug/fsolve.html](https://www.mathworks.com/help/optim/ug/fsolve.html)):

$e^{-e^{-(x_1 + x_2)}} = x_2 (1 + x_1^2)$

and

$x_1 \cos(x_2) + x_2 \sin(x_1) = 1/2$

It is not possible to solve either one for one variable in terms of the other. So instead, we will compute the objective function for a range of $x_1, x_2$ values, and then use a contour plot of each equation to see where there might be a solution.

The key to this visualization is where we draw the contours. A good choice is to highlight only the part of the solutions that bracket zero. Then we can see where they intersect, because there is probably a solution in that neighborhood.





In [ ]:
import numpy as np


def objective(X):
    x1, x2 = X
    z1 = np.exp(-np.exp(-(x1 + x2))) - x2 * (1 + x1**2)
    z2 = x1 * np.cos(x2) + x2 * np.sin(x1) - 0.5
    return np.array([z1, z2])


x1 = np.linspace(-10, 20)
x2 = np.linspace(-10, 20)

X1, X2 = np.meshgrid(x1, x2)

Z1, Z2 = objective([X1, X2])
plt.figure(figsize=(12, 8))
plt.contour(X1, X2, Z1, levels=np.linspace(-0.01, 0.01, 100))
plt.contour(X1, X2, Z2, levels=np.linspace(-0.01, 0.01, 100), cmap="jet")
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar();



In [ ]:
objective([0.4, 0.6])  # shows that the objective function is approximately zero



There is an intersection near $x_1=0.4$, and $x_2 = 0.6$. We can use that as an initial guess.



In [ ]:
from scipy.optimize import root

ans = root(
    objective, [2.5, 2.5]
)  # note we do not need ans, because ans will have two values in it.
ans, objective(ans.x)



In [ ]:
x1 = np.linspace(0, 1)
x2 = np.linspace(0, 1)

X1, X2 = np.meshgrid(x1, x2)
Z1, Z2 = objective([X1, X2])
plt.contour(X1, X2, Z1, levels=np.linspace(-0.1, 0.1, 100), cmap="seismic")
plt.contour(
    X1, X2, Z2, levels=np.linspace(-0.1, 0.1, 100), cmap="seismic", alpha=0.2
)  # set a colormap
plt.plot(*ans.x, "ro", ms=20)
plt.plot(*ans.x, "bs", ms=2)
plt.xlabel("$x_1$")
plt.ylabel("$x_2$")
plt.colorbar();



This shows the solution, and that the objective is practically equal to zero at that point.

You can see that trying to do this in more than 2 dimensions can quickly get difficult to visualize!





In [ ]:
# show that we only get one solution by brute force iteration over all the grid points
I, J = X1.shape
solns = []
for i in range(I):
    for j in range(J):
        x1, x2 = X1[i, j], X2[i, j]
        ans = root(objective, [x1, x2])
        solns += [ans.x]

solns = np.array(solns)

(np.mean(solns, axis=0), np.std(solns, axis=0))



In [ ]:
solns.shape



In [ ]:
50**10 * 16 / 1024 / 1024 / 1024  # Estimate of memory required to store 50**10 floats in an array



## Summary





-   We learned about a special class of nonlinear functions that are polynomials, and a series of useful functions to manipulate them.

-   We learned that you can use root to find solutions to coupled non-linear algebraic equations.

-   Next time, we will apply this to solving a nonlinear boundary value differential equation.



```{note}
Technically the polynomial functions here are not preferred anymore (https://numpy.org/doc/stable/reference/routines.polynomials.html). Instead, there is a new `numpy.polynomial` package that is preferred. This new package is more complete, and supposed to be more convenient. However, it introduces some surprising conventions, especially in polynomial fitting, that make it inconvenient in my opinion. That is why I do not recommend them here as the first entrypoint in using polynomials.

```



In [ ]:
from f22_06623 import MCQ
MCQ(lecture='08_nla_2')

